In [29]:
import os
import sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, os.path.join(cwd, "../"))

import pickle
import torch
import importlib
import numpy as np
import pandas as pd
import copy

import src.bin.tensorify as tensorify
import src.utils.data_conversion_utils as conversions
import src.data_manager.student_life_var_binned_data_manager as data_manager
import src.bin.trainer as trainer

from sklearn import metrics

from torch import nn
from copy import copy
from copy import deepcopy
from src import definitions
from src.bin import validations
from src.bin import statistics
from src.bin import plotting
from src.utils.read_utils import read_pickle
from src.utils import student_utils
from src.data_manager import sub_sampler
from src.data_manager import cross_val
from src.data_manager import helper as data_manager_helper
from tabulate import tabulate
from src.models import autoencoder
from src.models import multitask_learning
from src.models import user_dense_heads


pd.set_option('max_rows', 10000)

importlib.reload(validations)
importlib.reload(autoencoder)
importlib.reload(multitask_learning)
importlib.reload(user_dense_heads)
importlib.reload(sub_sampler)
importlib.reload(statistics)
importlib.reload(cross_val)
importlib.reload(definitions)
importlib.reload(conversions)
importlib.reload(tensorify)
importlib.reload(plotting)
importlib.reload(trainer)
importlib.reload(data_manager)
importlib.reload(student_utils)
importlib.reload(data_manager_helper)
feature_list = data_manager.FEATURE_LIST

# ##### Pickle #####
data = read_pickle('../data/training_data/shuffled_splits/training_date_normalized_shuffled_splits_select_features.pkl')

print(statistics.get_train_test_val_label_counts_from_raw_data(data))

      Train    Val    Test
--  -------  -----  ------
 0      111     45      29
 1      123     55      38
 2       93     30      21


In [41]:
import numpy as np
from sklearn.metrics import f1_score

previous_stress_labels = []
labels = []


# for key in data['data']:
#     if key in data['train_ids']:
#         continue
#     actual_data, missing_flags, time_delta, covariates, histogram, label = data['data'][key]
    
#     dow, sleep_rating, sleep_duration, previous_stress, exam_period = covariates
    
#     print("dow: {} sleep_rating: {} sleep_duration: {} previous_stress: {} label: {}".format(
#         dow, 
#         sleep_rating, 
#         sleep_duration, 
#         previous_stress, 
#         label))
    
#     previous_stress_labels.append(previous_stress)
#     labels.append(label)    
    
    
print("ValSet")
    
for key in data['val_ids']:
    actual_data, missing_flags, time_delta, covariates, histogram, label = data['data'][key]
    
    dow, sleep_rating, sleep_duration, previous_stress, exam_period = covariates
    
    print("dow: {} sleep_rating: {} sleep_duration: {} previous_stress: {} label: {}".format(
        dow, 
        sleep_rating, 
        sleep_duration, 
        previous_stress, 
        label))
    
    previous_stress_labels.append(previous_stress)
    labels.append(label)


ValSet
dow: 1.0 sleep_rating: 3.0 sleep_duration: 6.0 previous_stress: 1.0 label: 2.0
dow: 5.0 sleep_rating: 3.0 sleep_duration: 7.0 previous_stress: 1.0 label: 1.0
dow: 6.0 sleep_rating: 2.0 sleep_duration: 8.0 previous_stress: 0.0 label: 2.0
dow: 1.0 sleep_rating: 1.0 sleep_duration: 7.0 previous_stress: 0.0 label: 1.0
dow: 6.0 sleep_rating: 3.0 sleep_duration: 7.0 previous_stress: 1.0 label: 1.0
dow: 2.0 sleep_rating: 2.0 sleep_duration: 5.0 previous_stress: 1.0 label: 1.0
dow: 1.0 sleep_rating: 1.0 sleep_duration: 6.0 previous_stress: 2.0 label: 2.0
dow: 2.0 sleep_rating: 2.0 sleep_duration: 8.0 previous_stress: 1.0 label: 0.0
dow: 5.0 sleep_rating: 2.0 sleep_duration: 8.0 previous_stress: 2.0 label: 0.0
dow: 2.0 sleep_rating: 2.0 sleep_duration: 8.0 previous_stress: 2.0 label: 1.0
dow: 1.0 sleep_rating: 2.0 sleep_duration: 8.0 previous_stress: 1.0 label: 1.0
dow: 6.0 sleep_rating: 3.0 sleep_duration: 2.0 previous_stress: 2.0 label: 1.0
dow: 5.0 sleep_rating: 2.0 sleep_duration: 7.

In [42]:
previous_stress_labels, labels = np.array(previous_stress_labels), np.array(labels)
print(f1_score(y_true=labels, y_pred=previous_stress_labels, average='weighted'))

print(metrics.confusion_matrix(previous_stress_labels, labels))
# 0.4688043856595893
# print(0.6582988426251121 +  )

0.4688043856595893
[[22 20  1]
 [13 23 13]
 [10 12 16]]


In [38]:
from scipy.stats import mode
print(mode(labels))
most_common_predicions = np.full(130, 0.0)
print(f1_score(y_true=labels, y_pred=most_common_predicions, average='weighted'))

ModeResult(mode=array([1.]), count=array([93]))


ValueError: Found input variables with inconsistent numbers of samples: [218, 130]